# Name and ID: Hoang Tu Bui - 24005665

# The code executes till the end without any errors

# Original source: 

1. https://medium.com/@liyinxuan0213/step-by-step-double-deep-q-networks-double-dqn-tutorial-from-atari-games-to-bioengineering-dec7e6373896
1. https://stackoverflow.com/questions/26587527/cite-a-paper-using-github-markdown-syntax

# Why Reinforcement Learning is the ML paradigm of choice for this task?

Reinforcement Learning (RL) is well-suited for Atari games like Breakout where an agent must learn to make decisions through trial and error, optimizing its actions based on feedback in the form of rewards. This paradigm is distinct from supervised learning, which relies on pre-labeled data, and unsupervised learning, which seeks to find patterns without any labeled responses ([Shaheen et al., 2025](#shaheen2025reinforcementlearningstrategybasedatari)).

In Breakout, the agent must learn to control a paddle to bounce a ball and break bricks, receiving rewards only when bricks are broken. This setup presents a challenge where the agent must learn the consequences of its actions over time, making RL an ideal approach . ​

The success of RL in Breakout not only showcases its effectiveness in gaming but also highlights its potential in real-world applications where decision-making under uncertainty and learning from interaction are crucial.​

In summary, RL's framework aligns seamlessly with the challenges presented by Breakout, making it the machine learning paradigm of choice for such tasks.​

# The Environment

1. The game selected: **Breakout**

    The goal is to control a paddle in order to hit a ball toward brick walls at the top of the screen and to break them ([AtariAge - Atari 2600 Manuals (HTML) - Breakout (Atari), n.d.](#breakout-description)). The player starts with 5 lives and loses a life if the ball drops. The game terminates and resets when all lives are lost.

1. The input from the environment:

1. The control settings: four Discrete actions:
    - 0:NOOP (No Operation)
    - 1:FIRE
    - 2:RIGHT
    - 3:LEFT

In [5]:
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
import ale_py
import gymnasium as gym


gym.register_envs(ale_py)
env = gym.make("ALE/Breakout-v5", render_mode="human")
env = gym.wrappers.RecordEpisodeStatistics(env)
env = gym.wrappers.ResizeObservation(env, (84, 84))
env = gym.wrappers.GrayscaleObservation(env)
env = gym.wrappers.FrameStackObservation(env, 4)
env = MaxAndSkipEnv(env, skip=4)

spec = env.spec

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Action space: Discrete(4)
Observation space: Box(0, 255, (4, 84, 84), uint8)


In [ ]:
from torch import nn


class DQN(nn.Module):
    def __init__(self, nb_actions):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(4, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, nb_actions),
        )

    def forward(self, x):
        return self.network(x / 255.0)

In [ ]:
def Deep_Q_Learning(
    env,
    buffer_size=1_000_000,
    nb_epochs=30_000_000,
    train_frequency=4,
    batch_size=32,
    gamma=0.99,
    replay_start_size=50_000,
    epsilon_start=1,
    epsilon_end=0.1,
    exploration_steps=1_000_000,
    device="cuda",
    C=10_000,
    learning_rate=1.25e-4,
):

    # Initialize replay memory D to capacity N
    rb = ReplayBuffer(
        buffer_size,
        env.observation_space,
        env.action_space,
        device,
        optimize_memory_usage=True,
        handle_timeout_termination=False,
    )

    # Initialize action-value function Q with random weights
    q_network = DQN(env.action_space.n).to(device)
    # Initialize target action-value function Q_hat
    target_network = DQN(env.action_space.n).to(device)
    target_network.load_state_dict(q_network.state_dict())

    optimizer = torch.optim.Adam(q_network.parameters(), lr=learning_rate)

    epoch = 0
    total_rewards_list = []
    smoothed_rewards = []
    rewards = []
    total_loss_list = []
    loss_means = []
    losses = []
    best_reward = 0

    progress_bar = tqdm(total=nb_epochs)
    while epoch <= nb_epochs:

        dead = False
        total_rewards = 0

        # Initialise sequence s1 = {x1} and preprocessed sequenced φ1 = φ(s1)
        obs, _ = env.reset()

        for _ in range(random.randint(1, 30)):  # Noop and fire to reset environment
            obs, reward, terminated, truncated, info = env.step(1)

        while not dead:
            current_life = info["lives"]

            epsilon = max(
                (epsilon_end - epsilon_start) / exploration_steps * epoch
                + epsilon_start,
                epsilon_end,
            )
            if random.random() < epsilon:  # With probability ε select a random action a
                action = np.array(env.action_space.sample())
                # print("random")

            else:  # Otherwise select a = max_a Q∗(φ(st), a; θ)
                q_values = q_network(torch.Tensor(obs).unsqueeze(0).to(device))
                action = np.array(torch.argmax(q_values, dim=1).item())
                # print("not random")

            # Execute action a in emulator and observe reward rt and image xt+1
            next_obs, reward, terminated, truncated, info = env.step(action)
            dead = terminated or truncated

            # print(f"info: {info}")
            done = np.array(info["lives"] < current_life)

            # Set st+1 = st, at, xt+1 and preprocess φt+1 = φ(st+1)
            real_next_obs = next_obs.copy()

            total_rewards += reward
            reward = np.sign(reward)  # Reward clipping

            # Store transition (φt, at, rt, φt+1) in D
            rb.add(obs, real_next_obs, action, reward, done, info)

            obs = next_obs

            if epoch > replay_start_size and epoch % train_frequency == 0:
                # Sample random minibatch of transitions (φj , aj , rj , φj +1 ) from D
                data = rb.sample(batch_size)
                with torch.no_grad():
                    max_target_q_value, _ = target_network(data.next_observations).max(
                        dim=1
                    )
                    y = data.rewards.flatten() + gamma * max_target_q_value * (
                        1 - data.dones.flatten()
                    )
                current_q_value = (
                    q_network(data.observations).gather(1, data.actions).squeeze()
                )

                loss = F.huber_loss(y, current_q_value)

                # Perform a gradient descent step according to equation 3
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses.append(loss.item())

            # Every C steps reset Q_hat=Q
            if epoch % C == 0:
                target_network.load_state_dict(q_network.state_dict())

            epoch += 1
            if (epoch % 10_000 == 0) and epoch > 0:
                smoothed_reward = np.mean(rewards) if rewards else 0
                smoothed_rewards.append(smoothed_reward)
                total_rewards_list.append(rewards)
                rewards = []

                loss_mean = np.mean(losses) if losses else 0
                loss_means.append(loss_mean)
                total_loss_list.append(losses)
                losses = []

            if (epoch % 100_000 == 0) and epoch > 0:
                plt.plot(smoothed_rewards)
                plt.title("Average Reward on Breakout")
                plt.xlabel("Training Epochs [units of 10,000]")
                plt.ylabel("Average Reward per Episode")
                if not os.path.exists("./Imgs"):
                    os.makedirs("./Imgs")
                plt.savefig(f"./Imgs/average_reward_on_breakout_{epoch}.png")
                # plt.show()
                plt.close()

                plt.plot(loss_means)
                plt.title("Average Loss on Breakout")
                plt.xlabel("Training Epochs [units of 10,000]")
                plt.ylabel("Average Loss per Episode")
                if not os.path.exists("./Imgs"):
                    os.makedirs("./Imgs")
                plt.savefig(f"./Imgs/average_loss_on_breakout_{epoch}.png")
                # plt.show()
                plt.close()

                print(
                    f"Epoch: {epoch}, Loss: {loss_mean}, Smoothed Reward: {smoothed_reward}"
                )

            if epoch % 500_000 == 0 and epoch > 0:
                # if epoch % 10_000 == 0 and epoch > 0:
                checkpoint_path = f"./checkpoints/ddqn_checkpoint_{epoch}.pth"
                if not os.path.exists("./checkpoints"):
                    os.makedirs("./checkpoints")
                if not os.path.exists("./checkpoint_data"):
                    os.makedirs("./checkpoint_data")
                torch.save(q_network.state_dict(), checkpoint_path)

                # Save smoothed rewards
                with open(
                    f"./checkpoint_data/total_rewards_list_{epoch}.pkl", "wb"
                ) as f:
                    pickle.dump(total_rewards_list, f)

                # Save losses
                with open(f"./checkpoint_data/total_loss_list_{epoch}.pkl", "wb") as f:
                    pickle.dump(total_loss_list, f)

            progress_bar.update(1)
        rewards.append(total_rewards)

        if total_rewards > best_reward:
            best_reward = total_rewards
            if not os.path.exists("./best_models"):
                os.makedirs("./best_models")
            torch.save(q_network.cpu(), f"./best_models/best_model_{best_reward}")
            q_network.to(device)

# Test ref 
"...the **go to** statement should be abolished..." ([Dijkstra, 1968](#1)).




# References

1. <a id="1"></a> 
Dijkstra, E. W. (1968). 
Go to statement considered harmful. 
Communications of the ACM, 11(3), 147-148.

1. <a id="breakout-description"></a>
AtariAge—Atari 2600 Manuals (HTML)—Breakout (Atari). (n.d.). Retrieved May 5, 2025, from https://atariage.com/manual_html_page.php?SoftwareID=889

1. <a id="shaheen2025reinforcementlearningstrategybasedatari"></a>
Shaheen, A., Badr, A., Abohendy, A., Alsaadawy, H., & Alsayad, N. (2025). Reinforcement Learning in Strategy-Based and Atari Games: A Review of Google DeepMinds Innovations (No. arXiv:2502.10303). arXiv. https://doi.org/10.48550/arXiv.2502.10303